# The importance of signs
В этом задании мы вновь рассмотрим данные о пассажирах Титаника. Будем решать на них задачу классификации, в которой по различным характеристикам пассажиров требуется предсказать, кто из них выжил после крушения корабля.

#### Реализация в Scikit-Learn

В библиотеке scikit-learn решающие деревья реализованы в классах sklearn.tree.DecisionTreeСlassifier (для классификации) и sklearn.tree.DecisionTreeRegressor (для регрессии). Обучение модели производится с помощью функции fit.

Пример использования:

In [68]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([0, 1, 0])
clf = DecisionTreeClassifier()
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

В этом задании вам также потребуется находить важность признаков. Это можно сделать, имея уже обученный классификатор:

In [69]:
importances = clf.feature_importances_
importances

array([ 0.75,  0.25])

Переменная importances будет содержать массив "важностей" признаков. Индекс в этом массиве соответствует индексу признака в данных.

Стоит обратить внимание, что данные могут содержать пропуски. Pandas хранит такие значения как nan (not a number). Для того, чтобы проверить, является ли число nan'ом, можно воспользоваться функцией np.isnan.

Пример использования:

In [70]:
np.isnan(X)

array([[False, False],
       [False, False],
       [False, False]], dtype=bool)

#### Материалы

- [Подробнее про решающие деревья в sklearn](http://scikit-learn.org/stable/modules/tree.html)
- [Работа с пропущенными значениями в pandas](http://pandas.pydata.org/pandas-docs/stable/missing_data.html)
- [Подробнее о деревьях и их построении](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture07-trees.pdf)

#### Инструкция по выполнению

1. Загрузите выборку из файла titanic.csv с помощью пакета Pandas.
2. Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).
3. Обратите внимание, что признак Sex имеет строковые значения.
4. Выделите целевую переменную — она записана в столбце Survived.
5. В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.
6. Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).
7. Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над тем, чтобы убрать это ограничение.

In [71]:
import numpy as np
import pandas as pd

#### 1. Загрузите выборку из файла titanic.csv с помощью пакета Pandas.

In [101]:
data = pd.read_csv('titanic.csv', index_col='PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [73]:
print(data.columns)
print(data.shape)
print(data.dtypes)

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')
(891, 11)
Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object


#### 2. Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).

In [102]:
X = data[['Pclass', 'Fare', 'Age', 'Sex']]
X.head()

,Pclass,Fare,Age,Sex
PassengerId,,,,
1,3,7.2500,22.0,male
2,1,71.2833,38.0,female
3,3,7.9250,26.0,female
4,1,53.1000,35.0,female
5,3,8.0500,35.0,male


#### 3. Обратите внимание, что признак Sex имеет строковые значения.

In [103]:
mapping = {'male': 0, 'female': 1}
X = X.replace({ 'Sex': mapping })
X.head()

,Pclass,Fare,Age,Sex
PassengerId,,,,
1,3,7.2500,22.0,0
2,1,71.2833,38.0,1
3,3,7.9250,26.0,1
4,1,53.1000,35.0,1
5,3,8.0500,35.0,0


In [ ]:
X['Sex'] = X['Sex'].map(lambda x: 0 if x == 'female' else 1)

#### 4. Выделите целевую переменную — она записана в столбце Survived.

In [104]:
y = data['Survived']
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

#### 5. В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.

In [106]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans(X).head()

,Pclass,Fare,Age,Sex
PassengerId,,,,
6,3,8.4583,NaN,0
18,2,13.0000,NaN,0
20,3,7.2250,NaN,1
27,3,7.2250,NaN,0
29,3,7.8792,NaN,1


In [107]:
X = X[np.isfinite(X['Age'])]
X.head()

,Pclass,Fare,Age,Sex
PassengerId,,,,
1,3,7.2500,22.0,0
2,1,71.2833,38.0,1
3,3,7.9250,26.0,1
4,1,53.1000,35.0,1
5,3,8.0500,35.0,0


In [111]:
y = y[X.index.values]

In [110]:
print(len(X) == len(y))
print('Labels: ', len(X))
print('Samples: ', len(y))

True
Labels:  714
Samples:  714


#### 6. Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).

In [112]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=241)
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=241,
            splitter='best')

#### 7. Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

In [114]:
importances = clf.feature_importances_
importances

array([ 0.14751816,  0.29538468,  0.25658495,  0.30051221])

In [116]:
pd.Series(clf.feature_importances_)

0    0.147518
1    0.295385
2    0.256585
3    0.300512
dtype: float64

In [118]:
X.columns

Index(['Pclass', 'Fare', 'Age', 'Sex'], dtype='object')

Признаки: **Sex** и **Fare** являются наиболее информативными, так как при крушении корабля в первую очередь спасали женщин и пассажиров первого класса, где ехали пассажиры с наиболее дорогими билетами